## DDQN in Hockey Enviroment - Evaluation

Evaluate a trained DDQN against various environments

Base DQN implementation adapted from HW7

In [21]:
from importlib import reload

import numpy as np
import gymnasium as gym
from gymnasium import spaces
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

from DDQN.DQN import DQNAgent
from DDQN.DDQN import DDQNAgent
import hockey.hockey_env as h_env

In [22]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [ ]:
class DiscretizedHockey(h_env.HockeyEnv):
    def __init__(self, bins, **kwargs):
        super().__init__(**kwargs)

        self.bins = bins
        self.discretized_action_space = spaces.Discrete(self.bins)

### Normal Gameplay

#### Weak Opponent

In [ ]:
RENDER = True

reload(h_env)
#env = DiscreteActionWrapper(h_env.HockeyEnv(mode=h_env.Mode.NORMAL), bins=8)  # TODO: change bin count?
env = DiscretizedHockey(bins=5, mode=h_env.Mode.NORMAL)

env.reset()
if RENDER:
    env.render()

/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [ ]:
# TODO: load agent from file
DISCOUNT = 0.95
EPS = 0.2
UPDATE_TARGET_FREQ = 20
TAU = 1e-3

agent1 = DDQNAgent(
    env.observation_space,
    env.discrete_action_space,
    discount=DISCOUNT,
    epsilon=EPS,
    update_target_freq=UPDATE_TARGET_FREQ,
    tau=TAU
)

agent2 = h_env.BasicOpponent(weak=True)

In [ ]:
STEPS = 1000000

obs_buffer = []
reward_buffer = []
obs, _ = env.reset()
obs_agent2 = env.obs_agent_two()

for i in range(STEPS):
    if RENDER:
        env.render()
    a1 = agent1.act(obs)
    a1 = env.discrete_to_continous_action(a1)
    a2 = agent2.act(obs_agent2)

    obs, r, done, trunc, _ = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)

    obs_agent2 = env.obs_agent_two()

    if done or trunc:
        print(f"Episode done in {i} steps")


In [29]:
env.close()